In [1]:
import numpy as np
import pandas as pd
# from esmm import CTCVRNet
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
# from model_train import train_model

Init Plugin
Init Graph Optimizer
Init Kernel


In [5]:
## CTR 点击率 = 点击数/曝光数
## CVR 转化率 = 购买数/点击数
## CTCVR 点击转化率 = CVR*CTR = 购买数/曝光数

##初始化数据集

ctr_user_numerical_feature_train = pd.DataFrame(np.random.random((10000, 5)),
                                                columns=['user_numerical_{}'.format(i) for i in range(5)])
ctr_user_cate_feature_train = pd.DataFrame(np.random.randint(0, 10, size=(10000, 5)),
                                           columns=['user_cate_{}'.format(i) for i in range(5)])
ctr_item_numerical_feature_train = pd.DataFrame(np.random.random((10000, 5)),
                                                columns=['item_numerical_{}'.format(i) for i in range(5)])
ctr_item_cate_feature_train = pd.DataFrame(np.random.randint(0, 10, size=(10000, 3)),
                                           columns=['item_cate_{}'.format(i) for i in range(3)])

cvr_user_numerical_feature_train = pd.DataFrame(np.random.random((10000, 5)),
                                                columns=['user_numerical_{}'.format(i) for i in range(5)])
cvr_user_cate_feature_train = pd.DataFrame(np.random.randint(0, 10, size=(10000, 5)),
                                           columns=['user_cate_{}'.format(i) for i in range(5)])
cvr_item_numerical_feature_train = pd.DataFrame(np.random.random((10000, 5)),
                                                columns=['item_numerical_{}'.format(i) for i in range(5)])
cvr_item_cate_feature_train = pd.DataFrame(np.random.randint(0, 10, size=(10000, 3)),
                                           columns=['item_cate_{}'.format(i) for i in range(3)])


ctr_user_numerical_feature_val = pd.DataFrame(np.random.random((10000, 5)),
                                              columns=['user_numerical_{}'.format(i) for i in range(5)])
ctr_user_cate_feature_val = pd.DataFrame(np.random.randint(0, 10, size=(10000, 5)),
                                         columns=['user_cate_{}'.format(i) for i in range(5)])
ctr_item_numerical_feature_val = pd.DataFrame(np.random.random((10000, 5)),
                                              columns=['item_numerical_{}'.format(i) for i in range(5)])
ctr_item_cate_feature_val = pd.DataFrame(np.random.randint(0, 10, size=(10000, 3)), columns=['item_cate_{}'.format(i) for i in range(3)])

cvr_user_numerical_feature_val = pd.DataFrame(np.random.random((10000, 5)),
                                              columns=['user_numerical_{}'.format(i) for i in range(5)])
cvr_user_cate_feature_val = pd.DataFrame(np.random.randint(0, 10, size=(10000, 5)),
                                         columns=['user_cate_{}'.format(i) for i in range(5)])
cvr_item_numerical_feature_val = pd.DataFrame(np.random.random((10000, 5)),
                                              columns=['item_numerical_{}'.format(i) for i in range(5)])
cvr_item_cate_feature_val = pd.DataFrame(np.random.randint(0, 10, size=(10000, 3)),
                                         columns=['item_cate_{}'.format(i) for i in range(3)])

# print(ctr_user_numerical_feature_train.head())

ctr_target_train = pd.DataFrame(np.random.randint(0, 2, size=10000))
cvr_target_train = pd.DataFrame(np.random.randint(0, 2, size=10000))

ctr_target_val = pd.DataFrame(np.random.randint(0, 2, size=10000))
cvr_target_val = pd.DataFrame(np.random.randint(0, 2, size=10000))



In [8]:
train_data = [ctr_user_numerical_feature_train, ctr_user_cate_feature_train, ctr_item_numerical_feature_train,
              ctr_item_cate_feature_train, cvr_user_numerical_feature_train, cvr_user_cate_feature_train,
              cvr_item_numerical_feature_train, cvr_item_cate_feature_train, ctr_target_train, cvr_target_train]

val_data = [ctr_user_numerical_feature_val, ctr_user_cate_feature_val, ctr_item_numerical_feature_val,
            ctr_item_cate_feature_val, cvr_user_numerical_feature_val, cvr_user_cate_feature_val,
            cvr_item_numerical_feature_val, cvr_item_cate_feature_val, ctr_target_val, cvr_target_val]

In [15]:
##方便后续将类别型特征映射为统一长度的embedding向量嵌入。
cate_feature_dict = {}
user_cate_feature_dict = {}
item_cate_feature_dict = {}
for idx, col in enumerate(ctr_user_cate_feature_train.columns):
    cate_feature_dict[col] = ctr_user_cate_feature_train[col].max() + 1
    user_cate_feature_dict[col] = (idx, ctr_user_cate_feature_train[col].max() + 1)
for idx, col in enumerate(ctr_item_cate_feature_train.columns):
    cate_feature_dict[col] = ctr_item_cate_feature_train[col].max() + 1
    item_cate_feature_dict[col] = (idx, ctr_item_cate_feature_train[col].max() + 1)
print(cate_feature_dict)
print(user_cate_feature_dict)


{'user_cate_0': 10, 'user_cate_1': 10, 'user_cate_2': 10, 'user_cate_3': 10, 'user_cate_4': 10, 'item_cate_0': 10, 'item_cate_1': 10, 'item_cate_2': 10}
{'user_cate_0': (0, 10), 'user_cate_1': (1, 10), 'user_cate_2': (2, 10), 'user_cate_3': (3, 10), 'user_cate_4': (4, 10)}


In [12]:
#构建框架

import tensorflow as tf
# 是否使用GPU
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)
from tensorflow.keras.models import Model
from tensorflow.keras import layers

In [23]:
class CTCVRNet:
    def __init__(self, cate_feautre_dict):
        self.embed = dict()
        """
        将类别型特征做embedding嵌入，输入类别特征字典，键为特征名，值为类别的数量。
        统一映射到64维的embedding向量。
        """
        for k, v in cate_feautre_dict.items():
            self.embed[k] = layers.Embedding(v, 64)

    def build_ctr_model(self, ctr_user_numerical_input, ctr_user_cate_input, ctr_item_numerical_input,
	                    ctr_item_cate_input, ctr_user_cate_feature_dict, ctr_item_cate_feature_dict):
        """
        构建CTR预测模型
        """    
        user_embeddings, item_embeddings = [], []
        for k, v in ctr_user_cate_feature_dict.items():
            embed = self.embed[k](tf.reshape(ctr_user_cate_input[:, v[0]], [-1, 1]))
            embed = layers.Reshape((64,))(embed)
            user_embeddings.append(embed)
        for k, v in ctr_item_cate_feature_dict.items():
            embed = self.embed[k](tf.reshape(ctr_item_cate_input[:, v[0]], [-1, 1]))
            embed = layers.Reshape((64,))(embed)
            item_embeddings.append(embed)
        
        user_feature = layers.concatenate([ctr_user_numerical_input] + user_embeddings, axis=-1)
        item_feature = layers.concatenate([ctr_item_numerical_input] + item_embeddings, axis=-1)

        user_feature = layers.Dropout(0.5)(user_feature)
        user_feature = layers.BatchNormalization()(user_feature)
        user_feature = layers.Dense(128, activation='relu')(user_feature)
        user_feature = layers.Dense(64, activation='relu')(user_feature)

        item_feature = layers.Dropout(0.5)(item_feature)
        item_feature = layers.BatchNormalization()(item_feature)
        item_feature = layers.Dense(128, activation='relu')(item_feature)
        item_feature = layers.Dense(64, activation='relu')(item_feature)

        dense_feature = layers.concatenate([user_feature, item_feature], axis=-1)
        dense_feature = layers.Dropout(0.5)(dense_feature)
        dense_feature = layers.BatchNormalization()(dense_feature)
        dense_feature = layers.Dense(64, activation='relu')(dense_feature)
        pred = layers.Dense(1, activation='sigmoid', name='ctr_output')(dense_feature)
        return pred
        
    def build_cvr_model(self, cvr_user_numerical_input, cvr_user_cate_input, cvr_item_numerical_input,
	                    cvr_item_cate_input, cvr_user_cate_feature_dict, cvr_item_cate_feature_dict):
        """
        构建CVR预测模型
        """
        user_embeddings, item_embeddings = [], []
        for k, v in cvr_user_cate_feature_dict.items():
            embed = self.embed[k](tf.reshape(cvr_user_cate_input[:, v[0]], [-1, 1]))
            embed = layers.Reshape((64,))(embed)
            user_embeddings.append(embed)

        for k, v in cvr_item_cate_feature_dict.items():
            embed = self.embed[k](tf.reshape(cvr_item_cate_input[:, v[0]], [-1, 1]))
            embed = layers.Reshape((64,))(embed)
            item_embeddings.append(embed)
        user_feature = layers.concatenate([cvr_user_numerical_input] + user_embeddings, axis=-1)
        item_feature = layers.concatenate([cvr_item_numerical_input] + item_embeddings, axis=-1)

        user_feature = layers.Dropout(0.5)(user_feature)
        user_feature = layers.BatchNormalization()(user_feature)
        user_feature = layers.Dense(128, activation='relu')(user_feature)
        user_feature = layers.Dense(64, activation='relu')(user_feature)

        item_feature = layers.Dropout(0.5)(item_feature)
        item_feature = layers.BatchNormalization()(item_feature)
        item_feature = layers.Dense(128, activation='relu')(item_feature)
        item_feature = layers.Dense(64, activation='relu')(item_feature)

        dense_feature = layers.concatenate([user_feature, item_feature], axis=-1)
        dense_feature = layers.Dropout(0.5)(dense_feature)
        dense_feature = layers.BatchNormalization()(dense_feature)
        dense_feature = layers.Dense(64, activation='relu')(dense_feature)
        pred = layers.Dense(1, activation='sigmoid', name='cvr_output')(dense_feature)
        return pred
    def build(self, user_cate_feature_dict, item_cate_feature_dict):
		# CTR model input
        ctr_user_numerical_input = layers.Input(shape=(5,))
        ctr_user_cate_input = layers.Input(shape=(5,))
        ctr_item_numerical_input = layers.Input(shape=(5,))
        ctr_item_cate_input = layers.Input(shape=(3,))

        # CVR model input
        cvr_user_numerical_input = layers.Input(shape=(5,))
        cvr_user_cate_input = layers.Input(shape=(5,))
        cvr_item_numerical_input = layers.Input(shape=(5,))
        cvr_item_cate_input = layers.Input(shape=(3,))

        ctr_pred = self.build_ctr_model(ctr_user_numerical_input, ctr_user_cate_input, ctr_item_numerical_input,
                                        ctr_item_cate_input, user_cate_feature_dict, item_cate_feature_dict)
        cvr_pred = self.build_cvr_model(cvr_user_numerical_input, cvr_user_cate_input, cvr_item_numerical_input,
                                        cvr_item_cate_input, user_cate_feature_dict, item_cate_feature_dict)
        ctcvr_pred = tf.multiply(ctr_pred, cvr_pred)
        model = Model(
            inputs=[ctr_user_numerical_input, ctr_user_cate_input, ctr_item_numerical_input, ctr_item_cate_input,
                    cvr_user_numerical_input, cvr_user_cate_input, cvr_item_numerical_input, cvr_item_cate_input],
            outputs=[ctr_pred, ctcvr_pred])

        return model
    

In [24]:
import tensorflow as tf
import os
import time

tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import *


def train_model(cate_feature_dict, user_cate_feature_dict, item_cate_feature_dict, train_data, val_data):
	"""
	model train and save as tf serving model
	:param cate_feature_dict: dict, categorical feature for data
	:param user_cate_feature_dict: dict, user categorical feature
	:param item_cate_feature_dict: dict, item categorical feature
	:param train_data: DataFrame, training data
	:param val_data: DataFrame, valdation data
	:return: None
	"""
	ctcvr = CTCVRNet(cate_feature_dict)
	ctcvr_model = ctcvr.build(user_cate_feature_dict, item_cate_feature_dict)
	opt = optimizers.Adam(lr=0.003, decay=0.0001)
	ctcvr_model.compile(optimizer=opt, loss=["binary_crossentropy", "binary_crossentropy"], loss_weights=[1.0, 1.0],
	                    metrics=[tf.keras.metrics.AUC()])
	
	# keras model save path
	filepath = "esmm_best.h5"
	
	# call back function
	checkpoint = ModelCheckpoint(
		filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
	reduce_lr = ReduceLROnPlateau(
		monitor='val_loss', factor=0.8, patience=2, min_lr=0.0001, verbose=1)
	earlystopping = EarlyStopping(
		monitor='val_loss', min_delta=0.0001, patience=8, verbose=1, mode='auto')
	callbacks = [checkpoint, reduce_lr, earlystopping]
	
	# load data
	ctr_user_numerical_feature_train, ctr_user_cate_feature_train, ctr_item_numerical_feature_train, \
	ctr_item_cate_feature_train, cvr_user_numerical_feature_train, cvr_user_cate_feature_train, \
	cvr_item_numerical_feature_train, cvr_item_cate_feature_train, ctr_target_train, cvr_target_train = train_data
	
	ctr_user_numerical_feature_val, ctr_user_cate_feature_val, ctr_item_numerical_feature_val, \
	ctr_item_cate_feature_val, cvr_user_numerical_feature_val, cvr_user_cate_feature_val, \
	cvr_item_numerical_feature_val, cvr_item_cate_feature_val, ctr_target_val, cvr_target_val = val_data
	
	# model train
	ctcvr_model.fit([ctr_user_numerical_feature_train, ctr_user_cate_feature_train, ctr_item_numerical_feature_train,
	                 ctr_item_cate_feature_train, cvr_user_numerical_feature_train, cvr_user_cate_feature_train,
	                 cvr_item_numerical_feature_train,
	                 cvr_item_cate_feature_train], [ctr_target_train, cvr_target_train], batch_size=256, epochs=50,
	                validation_data=(
		                [ctr_user_numerical_feature_val, ctr_user_cate_feature_val, ctr_item_numerical_feature_val,
		                 ctr_item_cate_feature_val, cvr_user_numerical_feature_val, cvr_user_cate_feature_val,
		                 cvr_item_numerical_feature_val,
		                 cvr_item_cate_feature_val], [ctr_target_val, cvr_target_val]), callbacks=callbacks,
	                verbose=0,
	                shuffle=True)

In [25]:
ctcvr = CTCVRNet(cate_feature_dict)
ctcvr_model = ctcvr.build(user_cate_feature_dict, item_cate_feature_dict)
opt = optimizers.Adam(lr=0.003, decay=0.0001)
ctcvr_model.compile(optimizer=opt, loss=["binary_crossentropy", "binary_crossentropy"], loss_weights=[1.0, 1.0],
                    metrics=[tf.keras.metrics.AUC()])

train_model(cate_feature_dict, user_cate_feature_dict, item_cate_feature_dict, train_data, val_data)

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/devic

/Users/hewenpang/miniforge3/envs/tf/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replic

2024-01-02 22:56:19.099779: I tensorflow/core/common_runtime/placer.cc:114] args_0: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:19.099793: I tensorflow/core/common_runtime/placer.cc:114] range: (Range): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:19.099805: I tensorflow/core/common_runtime/placer.cc:114] RandomShuffle: (RandomShuffle): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:19.099808: I tensorflow/core/common_runtime/placer.cc:114] Identity: (Identity): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:19.099811: I tensorflow/core/common_runtime/placer.cc:114] identity_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:19.099815: I tensorflow/core/common_runtime/placer.cc:114] range/start: (Const): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:19.099817: I tensorflow/core/common_runtime/placer.cc:114] range/limit: (Const): /job:localhost/replica:0/task:0/device:C

Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in d

...

2024-01-02 22:56:46.012473: I tensorflow/core/common_runtime/placer.cc:114] args_0: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:46.012484: I tensorflow/core/common_runtime/placer.cc:114] range: (Range): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:46.012488: I tensorflow/core/common_runtime/placer.cc:114] Identity: (Identity): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:46.012491: I tensorflow/core/common_runtime/placer.cc:114] identity_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:46.012494: I tensorflow/core/common_runtime/placer.cc:114] range/start: (Const): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:46.012497: I tensorflow/core/common_runtime/placer.cc:114] range/limit: (Const): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:46.012500: I tensorflow/core/common_runtime/placer.cc:114] range/delta: (Const): /job:localhost/replica:0/task:0/device:CPU:0
2024-

Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0


2024-01-02 22:56:51.393047: I tensorflow/core/common_runtime/placer.cc:114] args_0: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:51.393060: I tensorflow/core/common_runtime/placer.cc:114] range: (Range): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:51.393064: I tensorflow/core/common_runtime/placer.cc:114] Identity: (Identity): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:51.393075: I tensorflow/core/common_runtime/placer.cc:114] identity_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:51.393078: I tensorflow/core/common_runtime/placer.cc:114] range/start: (Const): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:51.393081: I tensorflow/core/common_runtime/placer.cc:114] range/limit: (Const): /job:localhost/replica:0/task:0/device:CPU:0
2024-01-02 22:56:51.393084: I tensorflow/core/common_runtime/placer.cc:114] range/delta: (Const): /job:localhost/replica:0/task:0/device:CPU:0
2024-

Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_25001 in device /job:localhost/replica:0/task:0/device:GPU:0


KeyboardInterrupt: 